# Imports and Definitions

In [24]:


import numpy as np
import pandas as pd
import csv
import math
import pickle
import import_ipynb
import pattern_count
import time



def P1DominatedByP2(P1, P2):
    length = len(P1)
    for i in range(length):
        if P1[i] == -1:
            if P2[i] != -1:
                return False
        if P1[i] != -1:
            if P2[i] != P1[i] and P2[i] != -1:
                return False
    return True

# whether a pattern P is dominated by MUP M
def PDominatedByM(P, M):
    for m in M:
        if P1DominatedByP2(P, m):
            return True
    return False

# whether a pattern P dominates MUP M
def PDominatesM(P, M):
    for m in M:
        if P1DominatedByP2(m, P):
            return True
    return False

"""
# coverage of P among dataset D
def cov(P, D):
    cnt = 0
    for d in D:
        if P1DominatedByP2(d, P):
            cnt += 1
    return cnt

"""
def GenerateParents(P):
    parents = []
    length = len(P)
    for i in range(length):
        if P[i] != -1:
            q = P.copy()
            q[i] = -1
            parents.append(q)
    return parents

def GenerateByRule1(P, mcdes, attributes):
    children = []
    length = len(P)
    i = 0
    for i in range(length-1, -1, -1):
        if P[i] != -1:
            break
    if P[i] == -1:
        i -= 1
    for j in range(i+1, length, 1):
        for a in range(int(mcdes[attributes[j]]['min']), int(mcdes[attributes[j]]['max'])+1):
            s = P.copy()
            s[j] = a
            children.append(s)
    return children


def num2string(pattern):
    st = ''
    for i in pattern:
        if i != -1:
            st += str(i)
        st += '|'
    st = st[:-1]
    return st




def Prepatation(filename):
    mc = pd.read_csv(filename)
    mcdes = mc.describe()
    attributes = mcdes.columns.values
    return mc, mcdes, attributes




In [30]:

# check whether each pattern P is legal: cardinality and accuracy
def CheckWhetherMUP(P, Tha, Thc, pc_adult, pc_mc):
    adult_p = num2string(P)
    covdata = pc_adult.pattern_count(adult_p)
    covmc = pc_mc.pattern_count(adult_p)
    if covdata >= Thc:
        acc = 1 - covmc / covdata
        if acc < Tha:
            return True
    return False


def Deepdiver(mcfile, datafile, Tha, Thc):
    tao =  (1-Tha) * Thc
    mc, mcdes, attributes = Prepatation(mcfile)
    column_list_mc = np.array(mc.columns).tolist()
    pc_mc = pattern_count.PatternCounter(mcfile, column_list_mc, encoded=False)
    pc_mc.parse_data()

    data = pd.read_csv(datafile)
    column_list_adult = np.array(data.columns).tolist()
    pc_adult = pattern_count.PatternCounter(datafile, column_list_adult, encoded=False)
    pc_adult.parse_data()

    num_pattern_checked = 0

    root = [-1] * (len(attributes))
    S = [root]  # initial stack
    M = []  # maximal uncovered patterns
    while len(S) > 0:
        P = S.pop()
        print("pop from S: ", P)
        num_pattern_checked += 1
        if num_pattern_checked % 50000 == 0:
            print("num_pattern_checked = {}".format(num_pattern_checked))
        #print(P, len(M))
        if PDominatedByM(P, M):
            continue
        elif PDominatesM(P, M):
            uncoveredFlag = True
        else:
            cnt = pc_mc.pattern_count(num2string(P)) # cov(P, mclist)
            print("cnt = ", cnt)
            uncoveredFlag = (cnt < tao and cnt > 0) # add > 0 requirement here
        if uncoveredFlag:
            print(P)
            S1 = [P] # stack
            while len(S1) > 0:
                P1 = S1.pop()
                ParentNodes = GenerateParents(P1)
                for parent in ParentNodes:
                    cntparent = pc_mc.pattern_count(num2string(parent)) # cov(parent, mclist)
                    if cntparent < tao:
                        S1.append(parent)
                        break
                """
                if CheckWhetherMUP(P, Tha, Thc, pc_adult, pc_mc) is True:
                    M.append(P)
                    if len(M) % 10 == 0:
                        print(len(M))
                """
            # end while
        else:
            children = GenerateByRule1(P, mcdes, attributes)
            S = S + children
    return M, num_pattern_checked





In [31]:
whole_data_file = "../InputData/SmallDataset/SmallWhole_3_20.csv"
mis_class_data_file = "../InputData/SmallDataset/SmallMisClass_3_20.csv"
Tha = 0.4

Thc = 4

time1 = time.time()
M, num_pattern_checked = Deepdiver(mis_class_data_file, whole_data_file, Tha, Thc)
time2 = time.time()

print("time = {} seconds".format(time2-time1))
print(len(M), num_pattern_checked)

print(M)
print(num_pattern_checked)


pop from S:  [-1, -1, -1]
cnt =  8
pop from S:  [-1, -1, 15]
cnt =  1
[-1, -1, 15]
pop from S:  [-1, -1, 14]
cnt =  1
[-1, -1, 14]


KeyboardInterrupt: 